In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-18"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4637,2024-04-18,Polônia Liga De Basquete,12:30,Ostrow Wielkopolski,Lublin,1.36,3.03,177.5,1.85,1.87,0.0,0.00,0.00,QJXlGx2O,0.735294,0.330033,0.540541,0.534759,0.065327,158.130,25.700601,0.162528,1.8,1.643168,0.912871,169.10,1.750,0.304549,0.174028,2.0,161.488,34.170750,0.211599,1.8,1.643168,0.912871,184.80,1.680,0.382949,0.227946,31.0,89,110,1.90,1.68,125.758,174.802,0.537981,0.007603,NaN,0.47,0.094,3.829787,0.750341,0.6,-0.150341,-0.08,-0.016,-126.875000,0.502508,0.5,-0.002508
4638,2024-04-18,Nova Zelândia Nbl,04:30,Franklin Bulls,Otago Nuggets,1.43,2.68,181.5,1.85,1.81,0.0,0.00,0.00,6N4mDmgo,0.699301,0.373134,0.540541,0.552486,0.072435,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,101,105,1.18,2.48,0.000,0.000,0.430114,0.015456,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4639,2024-04-18,Noruega Blno,14:00,Fyllingen,Bærum,1.07,7.39,171.5,1.85,1.85,0.0,0.00,0.00,txoh8Jso,0.934579,0.135318,0.540541,0.540541,0.069897,113.638,20.663947,0.181840,3.0,0.000000,0.000000,109.62,1.226,0.149097,0.121613,74.0,375.630,186.406290,0.496250,1.2,1.643168,1.369306,263.53,4.514,1.929360,0.427417,-35.0,87,73,1.26,3.61,793.968,384.318,1.056481,0.000000,NaN,27.23,5.446,0.012853,0.000000,0.0,0.000000,9.23,1.846,3.461538,0.427936,0.6,0.172064
4640,2024-04-18,México Cibacopa,00:15,Ostioneros de Guaymas,Venados de Mazatlan,1.65,2.15,180.5,1.84,1.86,-2.5,1.83,2.22,4tTic9kI,0.606061,0.465116,0.543478,0.537634,0.071177,219.172,82.892006,0.378205,1.8,1.643168,0.912871,182.97,2.292,0.843428,0.367988,-2.0,207.844,72.723788,0.349896,2.4,1.341641,0.559017,219.24,2.306,0.864945,0.375085,16.0,107,108,1.71,2.03,238.242,277.260,0.186081,0.007644,0.136184,-1.66,-0.332,-1.957831,0.520749,0.6,0.079251,3.03,0.606,1.897690,0.359811,0.4,0.040189
4641,2024-04-18,Lituânia Lkl,12:50,Rytas,Siauliai,1.13,5.45,176.5,1.87,1.79,0.0,0.00,0.00,biEKsBs0,0.884956,0.183486,0.534759,0.558659,0.068442,168.798,66.535884,0.394175,2.4,1.341641,0.559017,281.58,1.880,0.986281,0.524617,36.0,199.546,104.862154,0.525504,1.2,1.643168,1.369306,165.44,2.484,1.752493,0.705512,-10.0,78,94,3.61,1.76,128.480,223.380,0.928481,0.030912,NaN,0.27,0.054,2.407407,0.000000,0.0,0.000000,-1.06,-0.212,-20.990566,0.408723,0.4,-0.008723
4642,2024-04-18,Lituânia Nkl,12:00,Suduva-Mantinga,BC Olimpas Palanga,1.48,2.53,156.5,1.85,1.85,0.0,0.00,0.00,dIIroyyG,0.675676,0.395257,0.540541,0.540541,0.070933,116.936,52.575356,0.449608,1.2,1.643168,1.369306,204.61,1.550,0.605145,0.390416,25.0,121.978,21.630488,0.177331,1.8,1.643168,0.912871,127.02,1.446,0.133903,0.092602,32.0,79,87,2.59,1.46,81.110,134.252,0.370305,0.000000,NaN,-3.83,-0.766,-0.626632,0.828593,0.4,-0.428593,-0.99,-0.198,-7.727273,0.685887,0.8,0.114113
4643,2024-04-18,Lituânia Lkl,12:30,Nevezis,Jonava,2.15,1.65,170.5,1.85,1.85,1.5,2.00,1.51,tG3PtVd6,0.465116,0.606061,0.540541,0.540541,0.071177,303.804,163.849269,0.539326,0.6,1.341641,2.236068,169.42,3.670,1.963683,0.535063,-39.0,231.130,83.801352,0.362572,1.8,1.643168,0.912871,231.47,2.896,1.042320,0.359917,18.0,86,79,1.97,2.93,260.438,492.962,0.186081,0.000000,0.197426,-1.80,-0.360,-3.194444,0.411899,0.4,-0.011899,7.63,1.526,0.425950,0.000000,0.0,0.000000


## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
4643,12:30,Lituânia Lkl,Nevezis,Jonava,2.15,Back Home
4659,21:00,Porto Rico Bsn,Piratas de Quebradillas,Indios de Mayaguez,1.56,Back Home
